In [1]:
import os, sys
sys.path.insert(0, '/home/jtorrenc/mvesc/ETL')
sys.path.insert(0, '/home/jtorrenc/mvesc/ModelsResults')
sys.path.insert(0, '/home/jtorrenc/mvesc/Features')

from mvesc_utility_functions import *
from estimate_prediction_model import build_outcomes_plus_features,temporal_cohort_test_split,impute_missing_values
import numpy as np
import pandas as pd
import random
from functools import partial
import itertools
from my_timer import Timer 
import pickle
import yaml
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def student_list(filename):
    model_name = filename.split('_')[-3]
    with open('/mnt/data/mvesc/Models_Results/pkls/'+filename+'_'+ model_name + '.pkl', "rb" ) as f:
        d = pickle.load(f)
    train = d['train_y'].index
    val = d['val_y'].index
    test = d['test_y'].index
    return train, val, test

In [3]:
def scale_features(train, val, test, strategy):
    """                                                                         
    Scales features based on the training values with the given strategy   
    Modified here to also return the scaler
                                                                                
    :param pd.DataFrame train:                                                  
    :param pd.DataFrame val:                                                    
    :param pd.DataFrame test:                                                   
    :param str strategy:                                                        
    :returns: scaled training, val, and test sets                               
    :rtype: pd.DataFrame, pd.DataFrame, pd.DataFrame                            
    """
    zero_variance_columns = [x for x in train.columns if len(train[x].unique()) == 1]
    train.drop(zero_variance_columns, axis=1, inplace=True)
    val.drop(zero_variance_columns, axis=1, inplace=True)
    test.drop(zero_variance_columns, axis=1, inplace=True)

    if (strategy == 'none'):
        return train, val, test

    elif(strategy == 'standard' or strategy == 'robust'):
        non_binary_columns = [x for x in train.columns if  len(train[x].unique())>2]
        if (len(non_binary_columns) > 0):
            scaler = StandardScaler() if strategy == 'standard' \
                     else RobustScaler()
            train_non_binary = train[non_binary_columns]
            val_non_binary = val[non_binary_columns]
            test_non_binary = test[non_binary_columns]
            scaler.fit(train_non_binary)
            train_non_binary = pd.DataFrame(scaler.transform(train_non_binary),
                columns = non_binary_columns, index = train.index)
            val_non_binary = pd.DataFrame(scaler.transform(val_non_binary),
                columns = non_binary_columns, index = val.index)
            test_non_binary = pd.DataFrame(scaler.transform(test_non_binary),
                columns = non_binary_columns, index = test.index)
            train_scaled = train.drop(non_binary_columns, axis=1)
            val_scaled = val.drop(non_binary_columns, axis=1)
            test_scaled = test.drop(non_binary_columns, axis=1)
            train_scaled = train_scaled.merge(train_non_binary,
                left_index=True, right_index=True)
            val_scaled = val_scaled.merge(val_non_binary,
                left_index=True, right_index=True)
            test_scaled = test_scaled.merge(test_non_binary,
                left_index=True, right_index=True)
            return train_scaled,val_scaled, test_scaled, scaler
        else:
            return train, val, test, scaler

    else:
        print('unknown feature scaling strategy. try "{}", "{}", or "{}"'\
              .format('standard', 'robust', 'none'))
        return train, val, test

In [4]:
def read_and_preprocess(filename):
    model_name = filename.split('_')[-3]
    with open('/mnt/data/mvesc/Models_Results/pkls/'+filename+'_'+ model_name + '.pkl', "rb" ) as f:
        d = pickle.load(f)
    model_options = d['model_options']
    model = d['estimator']
    columns = d['estimator_features']
    outcome_plus_features = build_outcomes_plus_features(model_options,None)
    outcome_plus_features.dropna(subset=[model_options['outcome_name'],
            model_options['cohort_grade_level_begin']], inplace=True)
    train, val, test = temporal_cohort_test_split(outcome_plus_features,
            model_options['cohort_grade_level_begin'],
            model_options['cohorts_test'],
            model_options['cohorts_val'],
            model_options['cohorts_training'])
    train_X_pre = train.drop([model_options['outcome_name'],
                      model_options['cohort_grade_level_begin']],axis=1)
    test_X_pre = test.drop([model_options['outcome_name'],
                        model_options['cohort_grade_level_begin']],axis=1)
    val_X_pre = val.drop([model_options['outcome_name'],
                      model_options['cohort_grade_level_begin']],axis=1)
    train_y = train[model_options['outcome_name']]
    test_y = test[model_options['outcome_name']]
    val_y = val[model_options['outcome_name']]
    train_X, val_X, test_X = impute_missing_values(train_X_pre, val_X_pre, test_X_pre,\
    model_options['missing_impute_strategy'])
    train_X = train_X.filter(columns)
    val_X = val_X.filter(columns)
    test_X = test_X.filter(columns)
    train_X, val_X, test_X, scaler = scale_features(train_X, val_X, test_X,
                                                    model_options['feature_scaling'])
    X = pd.concat([train_X,val_X,test_X])
    X_pre = pd.concat([train_X_pre,val_X_pre,test_X_pre])

    return X, X_pre, scaler, columns, model

In [5]:
def get_feature_scores(filename):
    with postgres_pgconnection_generator() as connection:
        with connection.cursor() as cursor:
            cursor.execute("""select feature,importance from model.feature_scores
                                    where filename = '{}'; """.format(filename))
            temp = cursor.fetchall()
    scores = pd.DataFrame(temp, columns = ['feature','importance'])
    scores.set_index('feature', inplace=True)
    return scores

In [6]:
def split_columns(X, columns):
    binary_features = pd.DataFrame(~X.apply(lambda x: len(x.unique()),axis=0).gt(2), columns = ['binary'])
    cts_columns = [c for c in columns if not binary_features['binary'].loc[c]]
    binary_columns = [c for c in columns if binary_features['binary'].loc[c]]
    return cts_columns, binary_columns

In [7]:
def categorical_feature_dict(all_features_path, binary_columns):
    with open(all_features_path, 'r') as f:
            all_features = yaml.load(f)
    feature_base = itertools.chain.from_iterable([a  for a in all_features.values()])
    feature_base = [y[:-1] if '*' in y else y for y in feature_base]
    binary_base = set()
    for f in feature_base:
        for b in binary_columns:
            if f in b and 'null' not in b:
                base = []
                flag=False
                if 'gr' in b:
                    for a in b.split('_'):
                        if flag:
                            base.append(a)
                            break
                        base.append(a)
                        if a=='gr':
                            flag=True
                    binary_base.add('_'.join(base))
                else:
                    binary_base.add(f)
    binary_base = list(binary_base)
    binary_dict = {}
    for b in binary_base:
        binary_dict[b] = [f for f in binary_columns if b == f[:len(b)]]
    return binary_dict

In [8]:
def plot_binary_features(model, student, binary_dict, train_X, plot):
    X_student = train_X.loc[student]
    I = pd.DataFrame(columns = ['delta','direction', 'current_val','var_type', 'was_null'], index=binary_dict.keys())
    
    for base, features in binary_dict.items():
        if len(features)>1: # for categorical
            X = pd.DataFrame(np.tile(X_student,[len(features),1]).transpose(),index=X_student.index) # reset X matrix
            current_val = 'other'
            for i,f in enumerate(features):
                if train_X[f].loc[student]:
                    current_val = f[len(base)+1:]
                for g in features:
                    X[i].loc[g] = 1 if g == f else 0
            if current_val == 'other': # for dropped features
                X[i+1] = X[i]
                for g in features:
                    X[i+1].loc[g] = 0
                current_val_ind = len(features)
                features.append(current_val)
            else:
                current_val_ind = [i for i,f in enumerate(features) if current_val in f][0]
            prob = [a[1] for a in model.predict_proba(X.transpose())]
            I['delta'].loc[base] = abs(np.mean([p - prob[current_val_ind] 
                                                 for i, p in enumerate(prob)
                                                if i != current_val_ind ]))
            I['direction'].loc[base] = 'protective' if prob[current_val_ind] < np.mean(prob) else 'risky'
            I['current_val'].loc[base] = current_val
            I['var_type'].loc[base] = 'categorical'
            I['was_null'].loc[base] = np.bool('null' in current_val)
            if plot:
                fig, ax = plt.subplots()
                plt.scatter( np.arange(len(features)), prob);
                plt.scatter( current_val_ind , prob[current_val_ind], label=current_val, color = 'red');
                ax.set_xticks( np.arange(len(features)))
                ax.set_xticklabels( [f[len(base)+1:] for f in features], rotation=90 ) ;
                plt.ylim([0,1])
                plt.title('{0} for student {1}'.format(base,student));
        else: # for truly binary features
            f= features[0]
            X = pd.DataFrame(np.tile(X_student,[2,1]).transpose(),index=X_student.index) # reset X matrix
            current_val = train_X[f].loc[student]
            X[0].loc[f] = 0 
            X[1].loc[f] = 1 
            prob = [a[1] for a in model.predict_proba(X.transpose())]
            diff = prob[int(current_val)] -  prob[int(not  bool(current_val))]
            I['delta'].loc[base] = abs(diff)
            I['current_val'].loc[base] = current_val
            I['var_type'].loc[base] = 'binary'
            I['was_null'].loc[base] = np.isnan(current_val) 
            if diff > 0:
                I['direction'].loc[base] = 'protective'   
            else:
                I['direction'].loc[base] = 'risky' 
            if plot:
                fig, ax = plt.subplots()
                plt.scatter( [0,1], prob);
                plt.scatter( current_val , prob[int(current_val)], label=current_val, color = 'red');
                ax.set_xticks([0,1] )
                ax.set_xticklabels( ['False', 'True'], rotation=90 ) ;
                plt.ylim([0,1])
                plt.title('{0} for student {1}'.format(f,student));
    return I

In [9]:
def plot_cts_features(model, student, cts_columns, train_X_pre, train_X, scaler):
    X_student = train_X.loc[student]
    n_steps = 100
    for feature in cts_columns:
        X = pd.DataFrame(np.tile(X_student,[n_steps,1]).transpose(),index=X_student.index) # reset X matrix
        current_val = train_X_pre[feature].loc[student]

        if not np.isnan(current_val): # skip null values
            values = np.linspace(min(train_X[feature]), max(train_X[feature]), n_steps)
            for i,v in enumerate(values):
                X[i].loc[feature] = v
            prob = [a[1] for a in model.predict_proba(X.transpose())]
            values_pre = pd.DataFrame(scaler.inverse_transform(X.loc[cts_columns].transpose()).transpose(),
                                index = cts_columns,
                                columns = X.columns)
            values_pre = values_pre.loc[feature]
            plt.figure()
            plt.hold='on'
            plt.vlines(current_val,0,1,label=str(current_val))
            plt.plot(values_pre, prob);
            plt.ylim([0,1])
            plt.xlim([min(values_pre)-1,max(values_pre)+1])
            #plt.legend()
            plt.title('{0} for student {1}'.format(feature,student));

In [10]:
def cts_feature_importance(model, student, cts_columns, train_X_pre, train_X, scaler, shift):
    X_student = train_X.loc[student]
    I = pd.DataFrame(columns = ['delta', 'direction', 'current_val','var_type','was_null'],index=cts_columns)
    for feature in cts_columns:
        s = shift*train_X[feature].std()
        X = pd.DataFrame(np.tile(X_student,[3,1]).transpose(),index=X_student.index) # reset X matrix
        X[0].loc[feature] = X[1].loc[feature] - s
        X[2].loc[feature] = X[1].loc[feature] + s
        prob = [a[1] for a in model.predict_proba(X.transpose())]
        down = prob[1]-prob[0]
        up = prob[1]-prob[2]
        I['delta'].loc[feature] = max(up,down)
        if prob[0] < prob[2]:
            I['direction'].loc[feature] = 'risky'
        else:
            I['direction'].loc[feature] = 'protective'
        I['current_val'].loc[feature] = X[1].loc[feature]
        I['was_null'].loc[feature] = np.isnan(train_X_pre[feature].loc[student])
    I['var_type'] = 'continuous'
    return I

In [11]:
def plot_features(model_filename, student, all_features_path):
    X, X_pre, scaler, column_order, model = read_and_preprocess(filename)
    cts_columns, binary_columns = split_columns(X, column_order)
    binary_dict = categorical_feature_dict(all_features_path, binary_columns)
    plot_cts_features(model, student, cts_columns, X_pre, X, scaler)
    plot_binary_features(model, student, binary_dict, X, True)

In [12]:
def get_feature_importances(model_filename, student, all_features_path, shift):
    X, X_pre, scaler, column_order, model = read_and_preprocess(filename)
    cts_columns, binary_columns = split_columns(X, column_order)
    binary_dict = categorical_feature_dict(all_features_path, binary_columns)
    I_cts = cts_feature_importance(model, student, cts_columns, X_pre, X, scaler, shift)
    I_binary = plot_binary_features(model, student, binary_dict, X, False)
    return pd.concat([I_cts,I_binary])

In [13]:
all_features_path = '../Features/all_features.yaml'
filename = '08_17_2016_grade_10_param_set_16_RF_jg_139'
train_students, val_students, test_students = student_list(filename)

In [14]:
val_students
student = 42556

In [15]:
# with Timer('feature importances') as t:
#     I = get_feature_importances(filename, student, all_features_path, 1) 


In [16]:
#I[np.logical_not(I['was_null'])].sort_values('delta', ascending=False).head()

In [19]:
with Timer('feature importances') as t:
    all_I = []
    c = 0
    for student in val_students:
        try:
            all_I.append(get_feature_importances(filename, student, all_features_path, 1))
        except:
            print('issue with student',student)
            raise
        if c%20:
            print('time elapsed: {0} sec (finished {1})'.format(t.time_check(),c))

issue with student 42985.0
feature importances: 2.7e+04 seconds elapsed


KeyboardInterrupt: 

In [22]:
len(all_I)

1368

In [ ]:
for student, I in zip(val_students[:len(all_I)], all_I):
    I.columns = [c+'_'+str(student) for c in I.columns]
I = all_I[0]
for I_next in all_I[1:]:
    I = I.join(I_next)

In [ ]:
#I[np.logical_not(I['was_null'])].sort_values('delta',ascending=False).head(10)

In [ ]:
#I_cts.sort_values('delta',ascending=False).head(10)

In [ ]:
#I_binary.sort_values('delta',ascending=False).head(10)

In [ ]:
#I_binary.sort_values('delta',ascending=False).tail(10)

In [ ]:
#plot_features(filename, student, all_features_path)